In [1]:
import numpy as np
import cv2

In [6]:
cap = cv2.VideoCapture(0)

face_detect_model = cv2.dnn.readNetFromCaffe('./model/deploy.prototxt.txt', './model/res10_300x300_ssd_iter_140000_fp16.caffemodel')

def face_detect_dnn(img):
    #step 1 Blob from the image
    blob = cv2.dnn.blobFromImage(img, 1, (300, 300), (104, 177, 123), swapRB=True)

    #step 2 set blob as input
    face_detect_model.setInput(blob)

    #step 3 get the output
    detections = face_detect_model.forward()

    #dwaring bound of box on top of the faces
    image = img.copy()
    h, w = img.shape[:2]

    for i in range(0, detections.shape[2]):
        confidence = detections[0,0,i,2]
        if confidence > 0.5:
            #diagonal points 3: 7
            box = detections[0, 0, i, 3:7] * np.array([w,h,w,h])
            box = box.astype('int')
            pt1 = (box[0], box[1])
            pt2 = (box[2], box[3])
            #draw circle
            cv2.rectangle(image, pt1, pt2, (0, 255, 0), 2)

            text = "score : {:.0f}%".format(confidence * 100)
            cv2.putText(image, text, pt1, cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255))

    return image

while True:
    ret, frame = cap.read()
    if ret == False:
        break
    img_detect = face_detect_dnn(frame)
    cv2.imshow('face', img_detect)
    if cv2.waitKey(1) == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()
